# Resource Estimates for Time evolution

In [83]:
# Let's read in the Carbon example provided by Sandia
import numpy as np
from mec_sandia.vasp_utils import read_vasp

ase_cell = read_vasp(f"../vasp_data/C_POSCAR")
# Next we can get some system paramters
volume_ang = ase_cell.get_volume()
print("Volume = {} A^3".format(volume_ang))

# To compute rs parameter we need volume in Bohr
from ase.units import Bohr
volume_bohr = volume_ang / Bohr**3
# and the number of valence electrons
num_carbon = len(np.where(ase_cell.get_atomic_numbers() == 6)[0])
# There is 1 hydrogen atom in the cell. Is this just a proton?
num_elec = 1 + num_carbon * 4
from mec_sandia.vasp_utils import compute_wigner_seitz_radius
# Get the Wigner-Seitz radius
rs = compute_wigner_seitz_radius(volume_bohr, num_elec)
print("rs = {} bohr".format(rs))
print("eta = {} ".format(num_elec))

Volume = 255.27133939199965 A^3
rs = 0.9289611708818379 bohr
eta = 513 


In [84]:
from mec_sandia.ft_pw_resource_estimates import pw_qubitization_costs 

num_bits_momenta = 6 # Number of bits in each direction for momenta
eps_total = 1e-3 # Total allowable error
num_bits_nu = 6 # extra bits for nu 
num_bits_nuc = 6 # extra bits for (quantum?) nuclear positions 
num_nuclei = 1# len(ase_cell.get_atomic_numbers) # Number of (quantum?) nuclei
toff, qubit = pw_qubitization_costs(np=num_bits_momenta, eta=num_elec, Omega=volume_bohr, eps=eps_total, nMc=8, nbr=8, L=num_nuclei)
print(f"Toffolis = {toff:4.3e}, qubits = {qubit}")

Toffolis = 7.360e+14, qubits = 10965.0


In [96]:
# Deuterium
import numpy as np
from mec_sandia.vasp_utils import read_vasp
import math

ase_cell = read_vasp(f"../vasp_data/D_POSCAR")
# Next we can get some system paramters
volume_ang = ase_cell.get_volume()
print("Volume = {} A^3".format(volume_ang))

# To compute rs parameter we need volume in Bohr
from ase.units import Bohr
volume_bohr = volume_ang / Bohr**3
# and the number of valence electrons
num_atoms = len(ase_cell.get_atomic_numbers())
print(ase_cell.)
# There is 1 hydrogen atom in the cell. Is this just a proton?
num_elec = num_atoms 
from mec_sandia.vasp_utils import compute_wigner_seitz_radius
# Get the Wigner-Seitz radius
rs = compute_wigner_seitz_radius(volume_bohr, num_elec)
print("rs = {} a0".format(rs))
print("eta = {} ".format(num_elec))
L_bohr = volume_bohr**(1.0/3.0)
print("L = {} a0".format(L_bohr))
from ase.units import Hartree
def num_pw_from_cutoff_ev(cutoff, length):
    factor = (2*np.pi / length)**2
    cutoff_ha = cutoff / Hartree
    n_squared = 2*cutoff_ha*factor / 3
    return n_squared**0.5 
num_pw = num_pw_from_cutoff_ev(1000, L_bohr)
print("Npw = {} ".format(num_pw))

Volume = 577.1514824733873 A^3
rs = 0.8132066816006787 a0
eta = 1729 
L = 15.733625397678654 a0
Npw = 1.9766514771634667 


In [106]:
from mec_sandia.ueg import UEG
import math

ueg = UEG.build(1730, 0.813, 230)
factor = (2*np.pi/ueg.box_length)
gmax = ueg.gvecs[-1]
ecut_ha = (1000/Hartree)
ecut_tilde = ecut_ha / (factor**2)
nmax = math.floor((2*ecut_tilde)**0.5)
num_pw_1d = (2*nmax - 1)
num_pw = num_pw_1d**3 
print("Npw_x = {}, Npw = {}, Npw_spherical = {}".format(num_pw_1d, num_pw, len(ueg.eigenvalues)))
print("np = {}".format(math.ceil(np.log2(num_pw**(1/3.)))))

Npw_x = 41, Npw = 68921, Npw_spherical = 41395
np = 6


In [108]:
from mec_sandia.ft_pw_resource_estimates import pw_qubitization_costs 

num_bits_momenta = 6 # Number of bits in each direction for momenta
eps_total = 1e-3 # Total allowable error
num_bits_nu = 6 # extra bits for nu 
num_bits_nuc = 6 # extra bits for (quantum?) nuclear positions 
num_nuclei = len(ase_cell.get_atomic_numbers()) # Number of (quantum?) nuclei
print(num_nuclei)
toff, qubit = pw_qubitization_costs(np=num_bits_momenta, eta=num_elec, Omega=volume_bohr, eps=eps_total, nMc=5, nbr=5, L=num_nuclei)
print(f"Toffolis = {toff:4.3e}, qubits = {qubit}")

1729
Toffolis = 1.936e+16, qubits = 32874.0


In [111]:
rs = 0.813
for factor in [1, 2, 4, 8, 10, 14, 20]:
    num_elec = 1729 // factor
    volume = (rs**3.0) * (4.0 * np.pi * num_elec / 3)
    toff, qubit = pw_qubitization_costs(np=num_bits_momenta, eta=num_elec, Omega=volume, eps=eps_total, nMc=5, nbr=5, L=num_nuclei)

864
432
216
172
123
86
